In [ ]:
%load_ext autoreload
%autoreload 2

# Evaluation

In [ ]:
from pathlib import Path 

import cdmetadl

root_dir = Path(cdmetadl.__file__).parent.parent
eval_dir = root_dir / f"output/full/eval"

In [ ]:
import pandas as pd
import plotly.express as px
from pathlib import Path

def read_df(path: Path) -> pd.DataFrame:
    df = pd.read_pickle(path)
    path_parts = path.relative_to(eval_dir).parts
    df.insert(0, 'Model', path_parts[2])
    df.insert(0, 'Mode', path_parts[3])
    return df

model_dirs = [eval_dir / f"train_cfg_baseline_k_{k}/eval_cfg_baseline_k_{k}" for k in [1, 5, 10, 20]]
full_df = pd.concat([read_df(filepath) for model_dir in model_dirs for filepath in model_dir.glob('**/evaluation.pkl') ])

for mode_name, df in full_df.groupby('Mode'):
    df_mean = df.groupby(['Number of Shots', 'Model'])['Normalized Accuracy'].mean().reset_index()
    
    fig = px.line(
        df_mean,
        x='Number of Shots',
        y='Normalized Accuracy',
        color='Model', 
        title=f'Normalized Accuracy vs. Number of Shots for Mode: {mode_name}',
        markers=True,  
    )

    fig.update_layout(
        xaxis_title="Number of Shots",
        yaxis_title="Normalized Accuracy",
        legend_title="Model"
    )

    fig.update_xaxes(
        tickvals=df_mean['Number of Shots'].unique(),
        range=[1, max(df_mean['Number of Shots'])]
    )
    
    fig.show()